# TinySMPC Tutorial

このチュートリアルでは、TinySMPCを使用して、安全なマルチパーティ計算の基本を説明します。

[セキュア・マルチパーティ計算](https://en.wikipedia.org/wiki/Secure_multi-party_computation) (SMPC)とは、複数の人が関数を計算できるようにする暗号技術で、関数そのものとその出力は公開されるが、入力は各人に非公開にされる。

簡単な例：友人グループが、個人の銀行残高を明らかにすることなく、グループの所有するお金の合計を計算したい。

## Setup

必要な3つのクラスだけをTinySMPCからインポートしよう。

TinySMPCは理解しやすいようにバニラPythonで書かれているので、外部モジュールは必要ない！

In [1]:
from tinysmpc import VirtualMachine, PrivateScalar, SharedScalar

## Creating Multiple Machines

まず、いくつかの `VirtualMachine` を作成しよう。

`VirtualMachine`はユーザのプライベートなコンピュータを表し、他のユーザに知られてはいけないデータが格納されている。

In [2]:
alice = VirtualMachine('alice')
bob = VirtualMachine('bob')
charlie = VirtualMachine('charlie')

Alice、Bob、Charlieを、あるネットワーク上で通信できる完全に独立したコンピュータと考える。

(簡単にするため、`VirtualMachines`はシミュレートされ、すべてこのノートブックカーネルでローカルに実行されます)。

次に、それぞれのコンピュータにプライベートデータを作りましょう。

## Generating Private Data

コンピュータ上のプライベートデータは `PrivateScalar` オブジェクトで表現される。これらの数字は各人の銀行残高だと考えてほしい。

In [3]:
a = PrivateScalar(40000, alice)
b = PrivateScalar(15000, bob)
c = PrivateScalar(-20000, charlie)  # Charlie is heavily in debt

アリスは自分の残高しか見ることができず、ボブとチャーリーの残高を見ることはできない。

このことは、アリスのコンピュータの中身を見ることで確認できる。(`alice`と表示すればいい！)

In [4]:
alice

VirtualMachine('alice')
 - PrivateScalar(40000, 'alice')

見てわかるように、アリスのコンピュータには自分の給料を表す`PrivateScalar`しか入っていない。

これはボブとチャーリーのコンピュータも同じです。

In [5]:
bob

VirtualMachine('bob')
 - PrivateScalar(15000, 'bob')

In [6]:
charlie

VirtualMachine('charlie')
 - PrivateScalar(-20000, 'charlie')

次に、これらのプライベート値に対してパブリック関数を計算する方法を見てみよう。

## Secret Sharing

アリス、ボブ、チャーリーの3人が、それぞれの銀行残高の合計を、誰にも知られずに計算したいとしよう。

これはまさに安全なマルチパーティ計算（SMPC）のために作られたものである。秘密分散」と呼ばれる技術は、[ほとんどの](https://crypto.stackexchange.com/questions/80764/is-there-a-secure-multi-party-computation-smpc-scheme-that-doesnt-use-secret) SMPCスキームの基本的な構成要素である。

秘密分散を行うには、`PrivateScalar`に対して `.share()` を呼び出します。

In [7]:
shared_a = a.share([alice, bob, charlie])
shared_a

SharedScalar
 - Share(3580162322096350479, 'alice', Q=None)
 - Share(-7552183910951222324, 'bob', Q=None)
 - Share(3972021588854911845, 'charlie', Q=None)

何が起こったのか？

まず、TinySMPCは加算秘密分散と呼ばれるテクニックを使って、暗号化された `a` の「シェア」を3つ生成した。上で見たように、各シェアは乱数のように見え、どのシェア（またはどの2つのシェア）からも`a`の真の値を見つけることは確かに不可能です。

次に、暗号化された共有をそれぞれのコンピュータに送りました。ボブのマシンを見ると、彼は新しい`Share`オブジェクト（アリスが彼に送ったもの）を所有しています。

In [8]:
bob

VirtualMachine('bob')
 - PrivateScalar(15000, 'bob')
 - Share(-7552183910951222324, 'bob', Q=None)

元の値 `a` に対応するすべての共有は `SharedScalar` オブジェクトで追跡される。

値 `a` を再構築するには、アリス、ボブ、チャーリーから3つの共有を持ってくる必要があります。

これは `SharedScalar` に対して `.reconstruct()` を呼び出すことで簡単に行うことができます。

In [9]:
shared_a.reconstruct(alice)

PrivateScalar(40000, 'alice')

裏では、TinySMPCはすべてのシェアをアリスに送り、`a`の元の値を再構築している。これは `.share()` の逆である。

これで加法的秘密共有の使い方がわかった。しかしこれだけではあまり役に立たないので、次は共有を使って計算を行う方法を見てみましょう。

(共有の生成と再構築の方法について興味があれば、[secret_sharing.py](https://github.com/kennysong/tinysmpc/blob/master/tinysmpc/secret_sharing.py)を参照してください)

## Basic Computation

加法的秘密共有スキームの不思議な特性は、暗号化された共有に対して直接演算ができることである！

これにより、安全な多人数計算における「計算」が可能になる。暗号化された`SharedScalar`で足し算をしてみよう。

In [10]:
shared_a = a.share([alice, bob, charlie])
shared_a = shared_a + 5000
shared_a.reconstruct(alice)

PrivateScalar(45000, 'alice')

うまくいった！暗号化された `SharedScalar` に直接 `5000` を追加したのですが、元の値に直接 `5000` を追加したのと同じ数値に復号化されたことに注意してください。

これを行うために、特別な演算プロトコルを各共有に直接適用した。面白いので、共有の値を直接調べてみよう。

In [11]:
shared_a = a.share([alice, bob, charlie])
print('Shares before adding 5000:')
print(shared_a)

shared_a = shared_a + 5000
print('\nShares after adding 5000:')
print(shared_a)

Shares before adding 5000:
SharedScalar
 - Share(-1228246268775828375, 'alice', Q=None)
 - Share(-5759674299707637981, 'bob', Q=None)
 - Share(6987920568483506356, 'charlie', Q=None)

Shares after adding 5000:
SharedScalar
 - Share(-1228246268775823375, 'alice', Q=None)
 - Share(-5759674299707637981, 'bob', Q=None)
 - Share(6987920568483506356, 'charlie', Q=None)


じっくり見ていると、最初の共有の暗号化された値に `5000` を直接追加しているのがわかるでしょう。

これはとても簡単で、[shared_addition.py](/tinysmpc/shared_addition.py)で実装を見ることができます。`SharedScalar`と(publicな)整数との足し算の他に、`SharedScalar`同士の足し算もできます。

このようにして、全員の銀行残高の合計を安全に計算することができます。

In [12]:
shared_a = a.share([alice, bob, charlie])
shared_b = b.share([alice, bob, charlie])
shared_c = c.share([alice, bob, charlie])

shared_sum = shared_a + shared_b + shared_c
shared_sum.reconstruct(bob)

PrivateScalar(35000, 'bob')

In [13]:
shared_sum = shared_a + shared_b
shared_sum.reconstruct(charlie)

PrivateScalar(55000, 'charlie')

ここでは、全員が所有するお金の合計を計算し、その結果の合計をボブに送った（ボブはその合計を「公に」共有することができる）。

足し算は暗号化された`SharedScalars`の間で直接行われ、誰も他の人の秘密の値を見ることはない！

次に、より高度な計算（乗算、比較、浮動小数点数など）の方法を見てみよう。

## Advanced Computation

足し算は楽しいが、SMPCを使えばもっと複雑な関数も計算できる。

### Multiplication

掛け算を試してみよう：

In [14]:
shared_a = PrivateScalar(5, alice).share([alice, bob])
shared_b = PrivateScalar(-10, bob).share([alice, bob])

shared_prod = 2 * shared_b * shared_a
shared_prod.reconstruct(alice)

PrivateScalar(-100, 'alice')

### Exponentiation

指数も同様に機能する（指数に公開整数を使用）：

In [15]:
shared_exp1 = shared_prod ** 2
shared_exp2 = shared_prod ** 3

print(shared_exp1.reconstruct(alice))
print(shared_exp2.reconstruct(bob))

PrivateScalar(10000, 'alice')
PrivateScalar(-1000000, 'bob')


### Comparison

公開整数との比較もできる：

In [16]:
print(shared_exp1 > 9999)
print(shared_exp2 > -999999)

PrivateScalar(1, 'p2')
PrivateScalar(0, 'p2')


(注意: ここで使用する SecureNN 比較アルゴリズムの詳細により、比較の出力はエフェメラルな `VirtualMachine` 上のブール値 `PrivateScalar` になります。ブール値は `.value` にアクセスすることで取得できます。どのように動作するかは、[shared_comparison.py](https://github.com/kennysong/tinysmpc/blob/master/tinysmpc/shared_comparison.py) を参照してください)。

これを利用して `SharedScalars` に ReLU 関数を実装できる可能性があります：

In [17]:
def relu(shared_x):
    if (shared_x > 0).value: 
        return shared_x
    return 0 * shared_x

shared_x = PrivateScalar(10, alice).share([alice, bob])

shared_relu_x = relu(shared_x)
shared_relu_neg_x = relu(-1 * shared_x)

print(shared_relu_x.reconstruct(alice))
print(shared_relu_neg_x.reconstruct(alice))

PrivateScalar(10, 'alice')
PrivateScalar(0, 'alice')


### Floating Point Numbers

最後に、実際のアプリケーションでは、浮動小数点数でSMPCを使いたいと思うかもしれません。これは可能です！

SharedScalar`と加法的秘密分散は一般的に整数でしか動作しないので、浮動小数点数を固定小数点整数表現に最初に変換する必要があることを思い出してください。

その方法を見てみよう。

In [18]:
from tinysmpc.fixed_point import fixed_point, float_point

In [19]:
fixed_point(0.005), fixed_point(-0.0005), fixed_point(0.00000005), fixed_point(-0.00000005)

(500000, -50000, 5, -5)

おわかりのように、TinySMPCは驚くほどシンプルな固定小数点表現を使っている。小数点以下8桁の精度を保つために、浮動小数点数を `10^8` 倍することで整数にマッピングしている。

浮動小数点数を取り戻すには、`float_point()`を使えばいい。

In [20]:
float_point(500000), float_point(-50000), float_point(5), float_point(-5)

(0.005, -0.0005, 5e-08, -5e-08)

では、実際に見てみよう：

In [21]:
a = fixed_point(1.2345)
b = fixed_point(-5.4321)

shared_a = PrivateScalar(a, alice).share([alice, bob])
shared_b = PrivateScalar(b, bob).share([alice, bob])
print(shared_a)
true_res = (-5.4321 - 1.2345) ** 2
shared_res = (shared_b - shared_a) ** 2
res = shared_res.reconstruct(alice)
res = float_point(res.value, n_mults=1)

res, true_res

SharedScalar
 - Share(-4115897540309803162, 'alice', Q=None)
 - Share(4115897540433253162, 'bob', Q=None)


(44.44355556, 44.44355556)

うまくいく！

大きな免責事項：固定小数点エンコーディングは、説明されていない `n_mults` パラメータからわかるように、動作が微妙です。変換係数やオーバーフローを注意深く考慮しなければ、正しくない結果を見ることになるでしょう。

優れたSMPCライブラリであれば、**透過的に**浮動小数点数を変換して処理してくれます。TinySMPCのソースコードをシンプルで読みやすいものにするために、この実装はやめました。

しかし、SMPCがどのように浮動小数点数を扱うことができるのか、その核心的なコンセプトはご理解いただけたと思います！

### Arbitrary Functions

これらのツールを使えば、より複雑な関数を計算することもできる。例えば、テイラー級数を使って `exp()` 関数を計算することができる。

これは読者のための練習として残しておこう。

## Limitations

TinySMPCはプロダクションSMPCライブラリになることを意図していないため、いくつかの重要な機能が省かれています。

 - SharedScalars`の分割。これは可能ですが、実装は複雑です（[SecureNN 論文](https://eprint.iacr.org/2018/442.pdf) のアルゴリズム 8 を参照してください）。
 - SharedScalars`の比較。これも可能だが、実装は複雑である。
 - SharedScalars`は浮動小数点数を透過的に扱えない。
 - ベクトル化の欠如、数学ライブラリの欠如など。

一般的なプロトコルとしてのSMPCの限界についても話す必要がある。

- SMPCは、すべての参加者が正直に正しい局所計算を適用することを前提としています。もしその気になれば、結果の値を変更することは些細なことです。これは「[honestly-but-curious](https://crypto.stanford.edu/pbc/notes/crypto/sfe.html)」仮定と呼ばれることもあります。完全性を提供する[SMPCの拡張](https://mortendahl.github.io/2017/08/13/secret-sharing-part3/)がある。
- これは、すべての参加者がオンラインであることを要求します。1台のマシンが故障すると、計算や復号を続けることができない。この問題を解決する拡張もある(「k-out-of-n secret sharing 」で検索)。
- ネットワークと同じくらい遅い！1回の乗算に、すべてのコンピュータ間で数回の往復メッセージが必要です。理想的には、マシンが高速なローカルネットワーク上に配置されていることが望ましいのですが、SMPCの実際の使用例の多くは、マシンが公共のインターネット上で分離されています。SMPCアルゴリズムは多くの場合、必要な通信量に基づいて評価されます。

## Recap

要約すると、SMPCは複数の人が関数を計算できるようにするもので、関数そのものとその出力は公開されるが、入力は各人に秘密にされる。

TinySMPCは、暗号化されたシークレットシェアを生成し、その暗号化されたデータに対して直接計算を行うという、コアとなるビルディングブロックを実装しています。

以下は、TinySMPCが提供する暗号化された操作の概要です。

|                    | Supported?              | Implementation                                                                          |
|--------------------|-------------------------|-----------------------------------------------------------------------------------------|
| **Addition**       | ✅                       | [SPDZ](https://eprint.iacr.org/2011/535.pdf) algorithm. <br/> See [shared_addition.py](https://github.com/kennysong/tinysmpc/blob/master/tinysmpc/shared_addition.py)             |
| **Subtraction**    | ✅                       | In terms of addition and multiplication.                                                 |
| **Multiplication** | ✅                       | [SPDZ](https://eprint.iacr.org/2011/535.pdf) algorithm.  <br/> See [shared_multiplication.py](https://github.com/kennysong/tinysmpc/blob/master/tinysmpc/shared_multiplication.py) |
| **Division**       | ❌ (too complicated)     | Possible with [SecureNN](https://eprint.iacr.org/2018/442.pdf).                                                                                       |
| **Exponentiation**       | ✅ (public integer only)     | In terms of multiplication.                                                                                       |
| **Greater Than**   | ✅ (public integer only) | [SecureNN](https://eprint.iacr.org/2018/442.pdf) algorithm. <br/> See [shared_comparison.py](https://github.com/kennysong/tinysmpc/blob/master/tinysmpc/shared_comparison.py)     |

次は何をする？

これはハイレベルなチュートリアルだった。これらのプロトコルが舞台裏でどのように動いているのかを理解したいのであれば、[TinySMPCのソースコード](https://github.com/kennysong/tinysmpc)をチェックすることをお勧めする！TinySMPCは、できるだけ小さく、理解しやすいように書かれています。

もしSMPCを実運用、特に機械学習に使いたいのであれば、[PySyft](https://github.com/OpenMined/PySyft/)と[TF Encrypted](https://github.com/tf-encrypted/tf-encrypted)を見ることをお勧めします。これらは、ベクトル化/テンソル化、高レベルの数学とMLのAPI、透過的な浮動小数点数変換などの便利な機能を実装しています。